# Video Processing for BT Lab
This notebook to test risky code fragments
<br>
<br>
![UofC logo](./pictures/uofc_logo-black.jpg)

In [1]:
#import libraries
import os
import cv2
import json
import re

In [ ]:
"""delete unwanted files"""
desired_path = r"E:\Christian\DI_centre_structured\DI_THERMAL_FLIR\3\2 meters\H\WOB\frames"
os.chdir(desired_path)
copied_files = [bmp_files for bmp_files in os.listdir() if "Copy" in bmp_files]
copied_files

In [ ]:
# for copy_file in copied_files:
#     os.remove(copy_file)

In [ ]:
"""
    A function to convert mp4 files to frames
"""

def convert_mp4_to_frames(mp4_file:str, output_dir:str):
    video = cv2.VideoCapture(mp4_file)
    os.makedirs(output_dir, exist_ok=True)
    frame_count = 0
    not_end_of_vid = True

    while not_end_of_vid:
        ret, frame = video.read()
        
        if not ret:
            not_end_of_vid = False
            break

        frame_file = os.path.join(output_dir, f"frame{frame_count:05d}.jpg")
        cv2.imwrite(frame_file, frame)
        frame_count += 1

    video.release()

    print(f"Conversion complete: {frame_count} frames saved in {output_dir}")